In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
from modelData import *
from gmsPython import nestingTree
from mEmissions import EmRegTargetsFromSYT, EmRegTargetsFromSYT_xt0

No clean-up of work-folder


# Create Model Data

With square adjustment costs.

## 0. Preliminaries

In [2]:
t0 = 2019

*This creates a copy of the IO data from the ```IOdata``` project into the local data folder*

In [3]:
ioName = f'IO{t0}_NCP'
f_IOdata = os.path.join(d['project'], 'IOdata','data','processedData', ioName)
shutil.copy(f_IOdata, os.path.join(d['data'], ioName))

'C:\\Users\\sxj477\\Documents\\GitHub\\CGE_Generator\\projects\\NationalClimatePolicy\\data\\IO2019_NCP'

*Define main settings:*

In [4]:
T  = t0 + 150 # set finite time horizon
name = 'vSimpleEOP_SAC' # Some global name that carries through all models/data to identify this version

*Load data:*

In [5]:
db = GpyDB(os.path.join(d['data'], ioName), name = f'{name}_{t0}', ws = d['work'])

*For relevant variables, add yearly index to data (this is currently all of them)*

In [6]:
def add_t0(k):
    k.index = stdSort(pd.MultiIndex.from_frame(k.index.to_frame(index=False).assign(t = t0)))
[add_t0(db(k)) for k in db.getTypes(['var','par'])];

*Add various time subsets that we'll rely on:*

In [7]:
addTimeToDB(t0, T, db)

## 0. Adhoc adjustments:

Create some adhoc adjustments to force a steady state without markups and (almost) no taxes.
1. Remove inventory investments entirely.
2. Set all taxes to zero except lump-sum taxes on households (this should finance government consumption)
3. Adjust depreciation rates to ensure steady state level of investments in baseline year.
4. Adjust household labor supply to ensure the household starts in steady state. Assume the household owns zero foreign assets, but all of the domestic shares.

#### Remove inventory and taxes:

Remove inventory and trade:

In [8]:
c = ('not', pd.Index(['itory'], name = 's'))
db['qD'] = adj.rc_pd(db('qD'), c)
db['d_qD'] = adj.rc_pd(db('d_qD'), c)

Set zero taxes:

In [9]:
db['TotalTax'] = adj.rc_pd(pd.Series(0, index = db('TotalTax').index), c, pm = False)
db['vTax']     = adj.rc_pd(pd.Series(0, index = db('vTax').index), c, pm = False)

#### Balance IO:

Return on household assets (given that it owns all domestic shares, no foreign ones):

In [10]:
returnOnAssets = (db('R_LR')-1)*adj.rc_pd(db('qD'), db('dur_p')).sum()

Then, the supply of labor is adjusted such that labor income and return on assets equals consumption + government consumption (as this will eventually be financed by taxes on the household). Note: This only works if there is a single government sector and a single household sector:

In [11]:
db.aom(pd.Series((db('qD').xs((t0, 'HH'), level = ['t','s']).sum()
                 +db('qD').xs((t0, 'G'), level = ['t','s']).sum()
                 -returnOnAssets),
                 index = adj.rc_pd(db('qS'), db('s_HH')).index),
       name = 'qS')

Adjust intermediate goods supply  to match demand:

In [12]:
db.aom(pd.Series(0, index = adj.rc_pd(db('qS'), db('s_i')).index)+adj.rc_pd(db('qD'), db('s_i')).groupby(['t','s']).sum(), name = 'qS')

*Adjust production sectors' demand for labor and intermediate goods. Do it by scaling up/down levels proportionally:*

In [13]:
TotalDemandForI = adj.rc_pd(db('qD'), db('inv_p')).groupby(['t','n']).sum()
TotalSupplyForI = adj.rc_pd(db('qS').droplevel('s'), db('inv_p'))
db.aom(stdSort(adj.rc_pd(db('qD'), db('inv_p')) * (TotalSupplyForI/TotalDemandForI)), name = 'qD')

Check demand for I_K versus I_K's own demand + its supply:

In [14]:
adj.rc_pd(db('qD'), db('s_i')).sum() == adj.rc_pd(db('qD'), db('inv_p')).sum() == adj.rc_pd(db('qS'), db('s_i')).sum()

True

*This assumes a single labor good:*

In [15]:
LIdx = pd.Index(['L'], name = 'n')
TotalDemandForL = adj.rc_pd(db('qD'), LIdx).groupby(['t','n']).sum()
TotalSupplyForL = adj.rc_pd(db('qS').droplevel('s'), LIdx)
db.aom(stdSort(adj.rc_pd(db('qD'), LIdx) * (TotalSupplyForL/TotalDemandForL)), name = 'qD')

Check that demand for labor = supply of labor:

In [16]:
db('qS').xs('L',level='n').sum() == db('qD').xs('L',level='n').sum()

True

*Define depreciation rate as the steady state rate:*

In [17]:
I0 = adjMultiIndex.applyMult(adj.rc_pd(adj.rc_pd(db('qD'), db('inv_p')), alias = {'n':'nn'}),
                             db('dur2inv'))
db.aom((I0/adj.rc_pd(db('qD'), db('dur_p'))).droplevel('nn'), name = 'rDepr')

*Update the durable price as well*

In [18]:
db.aom(db('rDepr')+db('R_LR')-1, name = 'pD_dur')

*Adjust supply to ensure zero profits with steady state level of investments:*

In [19]:
costs_nonDurables = adj.rc_pd(db('qD'), ('and', [db('s_p'), ('not', db('dur_p')), ('not', db('inv_p'))])).groupby(['t','s']).sum()
costs_Durables = (adj.rc_pd(db('qD'), db('dur_p')) * db('pD_dur')).groupby(['t','s']).sum()
db.aom(pd.Series(0, index = adj.rc_pd(db('qS'), db('s_p')).index)+costs_Durables+costs_nonDurables, name = 'qS')

*Adjust foreign sector's demand for domestic products to ensure equilibrium (check that this does not create negative values):*

In [20]:
SupplyDom = adj.rc_pd(db('qS'), db('s_p')).groupby(['t','n']).sum()
DemandDom_MinusTradeAndDurs = adj.rc_pd(db('qD'), ('and', [('not', db('s_f')), ('not', db('dur_p')), db('n_p')])).groupby(['t','n']).sum()
db.aom(stdSort(pd.Series(0, index = adj.rc_pd(db('qD'), db('s_f')).index) + SupplyDom-DemandDom_MinusTradeAndDurs), name = 'qD')

## 1. Production module

### 1.1. Nesting structure

Generally, we consider the following nesting structure:

In [21]:
mFull = pd.MultiIndex.from_tuples([('KELM', 'RxE'), ('KELM','KEL'), 
                                                    ('KEL','L'), ('KEL','KE'), 
                                                    ('KE','K'), ('KE','E'),
                                                    ('E','Energy_input'),('E','Energy_F')], names = ['n','nn'])

The materials nest excluding energy (```RxE```) is a CES nest of different final goods that compete with similar foreign ones. We use the syntax ```RxEym_x``` to denote the intermediate good that consists of ```x``` and the relevant foreign good:

In [22]:
nF_full = db('n_p')+'_F' # some of the set elements in the database may have been removed, because they are not used in a single sector
n_noEnergy = db('n_p').difference(['Energy']) 
nF_noEnergy= nF_full.difference(['Energy_F'])
RxE1 = pd.MultiIndex.from_product([['RxE'], 'RxEym_'+n_noEnergy], names = ['n','nn']) # mapping from RxE to intermediate goods
RxE2 = pd.MultiIndex.from_arrays([RxE1.get_level_values('nn'), n_noEnergy+'_input'], names = ['n','nn']) # mapping from intermediate good to domestic one 
RxE3 = pd.MultiIndex.from_arrays([RxE1.get_level_values('nn'), nF_noEnergy], names = ['n','nn']) # mapping from intermediate good to foreign one
mFull = reduce(pd.Index.union, [mFull, RxE1, RxE2, RxE3]) # Combine the four indices

If a sector uses inputs from all other sectors and foreign sectors, this is the mapping that describes its production structure. We allow for each sector to have a unique nesting structure by adding the sector index:

In [23]:
m = pyDatabases.cartesianProductIndex([db('s_p'), mFull])

Next, it may not be the case that all sectors rely on all types of inputs. In this case, we "trim" the nesting tree by eliminating the knots that are unused:

In [24]:
sparsity = adj.rc_pd(db('vD')[db('vD')!=0], db('s_p')).droplevel('t').index # what types of inputs do the sectors actually use in the data
# recall that inputs are called '_input' for products from the domestic sectors. Map the sparsity index to these:
mapNames = pd.Series(db('n_p')+'_input', index = db('n_p')).combine_first(
            pd.Series(sparsity.levels[-1], index = sparsity.levels[-1]))
sparsity = sparsity.set_levels(sparsity.levels[-1].map(mapNames), level = -1)

m = nestingTree.trimNestingStructure(m, sparsity)

Finally, we replace the upper-most level ```KELM``` with the name of the sector instead:

In [25]:
df = m.to_frame(index=False)
df.loc[df.n == 'KELM','n'] = df.loc[df.n == 'KELM', 's']
m = pd.MultiIndex.from_frame(df)
db['nestProduction'] = m

### 1.2. Elasticities

Elasticity of substitution between materials and ```qKEL```

In [26]:
sigmaOut = pd.Series(0.5, index = m[m.get_level_values('s')==m.get_level_values('n')].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```qKEL``` nest:

In [27]:
sigmaKEL = pd.Series(0.5, index  = m[m.get_level_values('n')=='KEL'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```KE``` nest:

In [28]:
sigmaKE = pd.Series(0.6, index  = m[m.get_level_values('n')=='KE'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```E``` nest:

In [29]:
sigmaE = pd.Series(0.5, index  = m[m.get_level_values('n')=='E'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution between materials:

In [30]:
sigmaRxE = pd.Series(0.1, index = m[m.get_level_values('n')=='RxE'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution import/domestic types:

In [31]:
sigmaRxEym = pd.Series(5, index = m[m.get_level_values('n').str.contains('RxEy')].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [32]:
sigma = pd.concat([sigmaOut, sigmaKEL, sigmaKE, sigmaE, sigmaRxE, sigmaRxEym], axis = 0 )
db.aom(sigma, name = 'sigma')

### 1.3. Regulation and emissions

Define relevant sector outputs and output shares (relevant if sectors have multiple outputs):

In [33]:
output = adj.rc_pd(db['qS'], db['s_p']) # output
inputs = adj.rc_pd(db['qD'], ('and', [db['s_p'], ('not', db['dur_p'])])) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares
db['qCO2'] = (db('qCO2') * outShares).dropna() # overwrite qCO2 to be split into potentially multiple outputs from each sector

Define implicit tax rate (this is a rough estimate of emission taxes, and its only available for $t\geq 1995$):

In [34]:
db['tauCO2'] = adjMultiIndex.applyMult((db('vTax').xs('Emissions',level='taxTypes') / db('qCO2').replace(0,1)).fillna(0), output.index)
db['dtauCO2']= db('tauCO2').index.droplevel('t').unique()
db['dqCO2']  = db('qCO2').index.droplevel('t').unique()

**Apply uniform CO2 taxes instead:**

Compute average tax and replace sector-specific ones:

In [35]:
db['tauCO2agg'] = (db('tauCO2') * db('qCO2')).groupby('t').sum() / (db('qCO2').groupby('t').sum()) # average CO2 tax
db['tauCO2'] = pd.Series(0, index = db('tauCO2').index)+db('tauCO2agg')

Compute totals for emission taxes and replace in ```vTax```:

In [36]:
vTax_Emissions = pyDatabases.pdSum((db('tauCO2') * db('qCO2').replace(0,1)), 'n')
vTax_Emissions_= vTax_Emissions.rename('vTax').reset_index().assign(taxTypes = 'Emissions').set_index(db['vTax'].domains)['vTax']
db.aom(vTax_Emissions_, name = 'vTax', priority = 'second')

For the residual part add as a lump-sum transfer:

In [37]:
db.aom(adj.rc_pd(db('TotalTax'), db['s_p'])-db('vTax').xs('Emissions',level='taxTypes'), name = 'tauLump')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')

Define sector-specific prices given regulation:

In [38]:
db.aom(stdSort(adj.rc_pd((db('tauD')+db('p')).dropna(), inputs)), name = 'pD')

Add the value of the "index fund" to be equal to the capital stock:

In [39]:
db['vIdxFund'] = adj.rc_pd(db('qD'), db('dur_p')).groupby('t').sum()

## 2. Investment module

### 2.1. Nesting structure

The investment module consists only of the materials nest, but includes energy:

In [40]:
dImport = adj.rc_pd(db('dImport'), db('s_i'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_i'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_i'))

*Add mappings:*

In [41]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'Materials_'+x['n'].astype(str), u = 'Materials')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [42]:
df = dImport_dom.to_frame(index=False).assign(u = 'Materials')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [43]:
df = dImport_for.to_frame(index=False).assign(u = 'Materials')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level (Materials) with the name of the sector:

In [44]:
df = m.to_frame(index=False)
df.loc[df.n == 'Materials','n'] = df.loc[df.n == 'Materials', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [45]:
db['nestInvestment'] = m

### 2.2: Elasticities:

In [46]:
sigmaI = pd.Series(.5, index = m.droplevel('nn').unique(), name = 'sigma')
db.aom(sigmaI, name = 'sigma')

### 2.3. Regulation:

The investment sector does not itself emit emissions; however, investments are still tied to emissions through this sectors' reliance on inputs from other sectors (that are emission intensive).

In [47]:
output = adj.rc_pd(db['qS'], db['s_i']) # output
inputs = adj.rc_pd(db['qD'], db['s_i']) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares

There rest is added as a lump sum tax:

In [48]:
db.aom(adj.rc_pd(db('TotalTax'), db['s_i']), name = 'tauLump')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')

Define sector-specific prices given regulation:

In [49]:
db.aom(stdSort(adj.rc_pd(db('tauD')+db('p'), inputs)), name = 'pD')

## 3. Households

### 3.1. Nesting structure

The household consumption nesting structure is somewhat similar to the investment sectors, with the exception that the top nest is a consumption aggregate that captures intertemporal preferences for consumption smoothing.

In [50]:
dImport = adj.rc_pd(db('dImport'), db('s_HH'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_HH'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_HH'))

*Add mappings:*

In [51]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'C_'+x['n'].astype(str), u = 'C')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [52]:
df = dImport_dom.to_frame(index=False).assign(u = 'C')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [53]:
df = dImport_for.to_frame(index=False).assign(u = 'C')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level with the name of the household consumption aggregate:

In [54]:
df = m.to_frame(index=False)
df.loc[df.n == 'C','n'] = 'C_'+df.loc[df.n == 'C', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [55]:
db['nestHH'] = m

Add mapping from consumption aggregate to labor:

In [56]:
db['L2C'] = pd.MultiIndex.from_arrays([db('s_HH'), pd.Index(['L']*len(db('s_HH')), name = 'n'), ('C_'+db('s_HH')).rename('nn')])

### 3.2. Elasticities/preferences:

Upper level nest:

In [57]:
sigma_HH_upper = pd.Series(0.5, index = m[m.get_level_values('n') == 'C_HH'].droplevel('nn').unique(), name = 'sigma')

Lower-level (import/domestic competition):

In [58]:
sigma_HH_Import = pd.Series(2, index = m[m.get_level_values('n') != 'C_HH'].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [59]:
sigma_HH = pd.concat([sigma_HH_upper, sigma_HH_Import], axis = 0) 
db.aom(sigma_HH, name = 'sigma')

Frisch elasticity, CRRA, time preferences:

In [60]:
db.aom(pd.Series(0.1, index = db('s_HH')), name = 'frisch')
db.aom(pd.Series(2, index = db('s_HH')), name = 'crra')
db.aom(pd.Series(1/db('R_LR'), index = db('s_HH')), name = 'discF')

### 3.3. Regulation and assets

*Adhoc regulation: Fix lump sum taxation to make sure that it finances government consumption* 

In [61]:
output = adj.rc_pd(db('qS'), db['s_HH'])
inputs = adj.rc_pd(db('qD'), db['s_HH'])
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(adj.rc_pd(db('TotalTax'), db('s_HH'))+db('qD').xs('G',level='s').sum(), name = 'TotalTax')
db.aom(adj.rc_pd(db('TotalTax'), db('s_HH')), name = 'tauLump')

*Define sector-specific prices given regulation:*

In [62]:
db.aom(stdSort(adj.rc_pd((1+db('tauD'))*db('p'), inputs)), name = 'pD')
db.aom(stdSort(adj.rc_pd(db('p')*(1-db('tauS')), output)), name = 'pS') # in this case pS is the after-tax wage rate

*Add asset value:*

In [63]:
db.aom(pd.Series(1, index = db('s_HH')), name = 'uIdxFund') # share of assets in domestic shares
db.aom(pd.Series(0, index = db('s_HH')), name = 'vA_F') # value of foreign assets
db.aom(pd.Series(db('vIdxFund').xs(t0), index = pd.MultiIndex.from_product([db('t'), db('s_HH')])), name = 'vA')

## 4. Government

### 4.1. Nesting structure

Government consumption is nested in a similar way as household consumption - and uses the same elasticities:

In [64]:
dImport = adj.rc_pd(db('dImport'), db('s_G'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_G'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_G'))

*Add mappings:*

In [65]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'GC_'+x['n'].astype(str), u = 'GC')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [66]:
df = dImport_dom.to_frame(index=False).assign(u = 'GC')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [67]:
df = dImport_for.to_frame(index=False).assign(u = 'GC')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level with the name of the household consumption aggregate:

In [68]:
df = m.to_frame(index=False)
df.loc[df.n == 'GC','n'] = 'GC_'+df.loc[df.n == 'GC', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [69]:
db['nestG'] = m

### 4.2. Elasticities

Upper level nest:

In [70]:
sigma_G_upper = pd.Series(.9, index = m[m.get_level_values('n') == 'GC_G'].droplevel('nn').unique(), name = 'sigma')

Lower-level (import/domestic competition):

In [71]:
sigma_G_Import = pd.Series(2, index = m[m.get_level_values('n') != 'GC_G'].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [72]:
sigma_G = pd.concat([sigma_G_upper, sigma_G_Import], axis = 0) 
db.aom(sigma_G, name = 'sigma')

### 4.3. Regulation

The government regulates/taxes itself (because data says so). Because we use incomplete data on taxes (especially on household taxes/transfers), we make some adhoc adjustments here:
* Remove taxes on inventory (because the "inventory sector" is only used for completeness of the IO system).
* Set flat VAT tax rate on government consumption to target taxes from government sector.

In [73]:
inputs = adj.rc_pd(db('qD'), db('s_G'))
db['TotalTax'] = adj.rc_pd(db('TotalTax'), ('not', pd.Index(['itory'],name='s'))) # remove inventory taxes
db['d_TotalTax'] = db['TotalTax'].index.droplevel('t').unique() # what sectors pay taxes
rVAT = adj.rc_pd(db('TotalTax'), db('s_G')) / pyDatabases.pdSum(inputs,'n') # define VAT rate of taxes
db.aom(adjMultiIndex.applyMult(rVAT,inputs.index), name = 'tauD') # add flat rate VAT on all demand
db.aom(stdSort(adj.rc_pd(db('tauD')+db('p'), inputs)), name = 'pD')

## 5. Trade

Set export elasticity to 7.5:

In [74]:
db.aom(pd.Series(7.5, index = db('dExport')), name='sigma')

*Note: We should add some information on regulation on trade at some point. Here, just add flat tariff/subsidy on all exports:* 

In [75]:
avgTariffRate = adj.rc_pd(db('TotalTax'), db('s_f'))/pyDatabases.pdSum(adj.rc_pd(db('qD'), db('s_f')), 'n')
TariffRate = stdSort(adjMultiIndex.applyMult(avgTariffRate, db('dExport')))
db.aom(TariffRate, name = 'tauD')
db.aom(pd.Series(0, index = adj.rc_pd(db('TotalTax'), db('s_f')).index), name = 'tauLump')
db.aom(stdSort(adj.rc_pd(db('tauD')+db('p'), db('s_f'))), name = 'pD')

## 6. Emissions

### 6.1. EOP abatement

Create abatement technology toy data. This creates a number of CCS-like technologies with costs decreasing over time at a rate $g$ to some year (here 2050):

In [76]:
n, Ttech = 6, 2050 # number of techs, what year do we "stop" technical progress 
c0 = np.hstack([6000, np.linspace(4000, 2000, n-1)]) # costs in baseline year
pot0 = np.hstack([.5, np.full(n-1, .25)]) # potential for reduction
g = .02 # growth rate in costs (declines exponentially)
db['gAbateTech'] = pd.Series(g, index = db('t'))
costs = pd.DataFrame((np.power(1-g, range(Ttech-t0+1)).reshape(Ttech-t0+1,1) * c0 ), 
                     index = pd.Index(range(t0, Ttech+1), name = 't'), 
                     columns = 'CCS'+pd.Index(range(n), name = 'tech').astype(str)).stack().swaplevel('t','tech')
pots = pd.DataFrame(np.tile(pot0, (len(db('t')), 1)), index = db('t'), columns = costs.index.levels[0]).stack().swaplevel('t','tech')
DACCosts= pd.Series(10000, index = db('t')) # keep costs high for DAC in this run

Extrapolate costs for CCS types to cover the entire time horizon (constant costs after 2050), adjust scale of variables (adjust unit from DKK/ton to billion DKK / M1990 level):

In [77]:
db['techCost'] = extrapolateUpper(costs, T) * (db('M1990') * 1e6)/db._scale
db['techPot'] = extrapolateUpper(pots, T)
db['DACCost'] = DACCosts * (db('M1990') * 1e6)/db._scale
db['tech'] = costs.index.levels[0]
db['qCO2Base'] = 0 # remove DAC potential

Next, define average CO2 tax, create ```tauDist``` that measures relative weight of CO2 in a specific sector:

In [78]:
db['uCO2'] = adj.rc_pd(db('qCO2'), db('s_p')) / adj.rc_pd(db('qS'), db('s_p')) # co2 share
db['tauCO2agg'] = (db('tauCO2') * db('qCO2')).groupby('t').sum() / (db('qCO2').groupby('t').sum()) # average CO2 tax
db['tauDist'] = pd.Series(1, index = db('tauCO2').index)
# db['tauDist']   = db('tauCO2')/db('tauCO2agg')
db['qCO2agg'] = db('qCO2').groupby('t').sum()

For now, define the effective tax rate used in production modules as the copy of true tax:

In [79]:
db['tauEffCO2'] = db('tauCO2').copy()

Add technology smoothing parameters (adhoc levels for now):

In [80]:
db.aom(pd.Series(db('techCost').min()/5, index = db('t')), name = 'DACSmooth')
db.aom(pd.Series(db('techCost').min()/5, index = db('techPot').index), name = 'techSmooth')

### 6.2. Emission regulation

*Add some emission targets (this relies on ```_xt0``` method, which removes the baseline year from regulation scheme. This is because abatement capital is fixed in the baseline year and thus there is no scope for additional abatement in this year)*

In [81]:
targets = pd.Series([0], index = pd.Index([2050], name = 't'))
[db.aom(v, name = k) for k,v in EmRegTargetsFromSYT_xt0(targets, db('t'), db('qCO2agg').xs(t0)).items()];

**NOTE:** It is important for the subsequent code that the emission budget target (```qCO2_EB[t]```) defines the cumulative target in the *last* year of the relevant time horizon.

## 7. Globals

Other parameters/variables used throughout:

In [82]:
db.aom(pd.Series(db('R_LR'), index = db('t')), name = 'Rrate', priority = 'first') # fix interest rate path at long run level
db.aom(pd.Series(db('R_LR')*(1+db('infl_LR')), index = db('t')), name = 'iRate', priority = 'first')

## Export

In [83]:
db.export(repo=d['data'])